## 18.1 The Log-Likelihood Gradient

### pp. 607 conditions for MCMC estimation of partition function for continuous distribution

This talks about assumption on Eq. (18.14).

> This identity is applicable only under certain regularity conditions on ... Fortunately, most machine learning models of interest have these properties.

I don't think the third condition here may be satisfied for all useful models, since it requires a upper bound $R(x)$ for all $\theta$, and this upper bound may depend on $\theta$. However, I think there are two ways to fix it, as long as for any interval (or cartesian product of interval, for high dimensional case) of $\theta$, we can find such a $R(x)$. 1) In practice, $\theta$ can't be unbounded, since computer has finite precision, and we can think the possible range of $\theta$ as in something like $[-10^{200}, 10^{200}]$; 2) check [here](./chapter_18/LeibnizRule.pdf) (original link <https://people.hss.caltech.edu/~kcb/Notes/LeibnizRule.pdf> or [here](./chapter_18/38599.pdf) (original link <http://planetmath.org/sites/default/files/texpdf/38599.pdf>), where weaker conditions are given and should be satisified by all interesting models.

Intuitively, I really think we don't need to find a $R(x)$ for all $\theta$, because Eq. (18.14) itself only cares about $\theta$ in a small range around current value. So those weaker conditions make sense to me.

The original set of conditions is also stated in <https://en.wikipedia.org/wiki/Leibniz_integral_rule#Measure_theory_statement>

## 18.2 Stochastic Maximum Likelihood and Contrastive Divergence

### pp. 608 essentials of approximate MCMC

> The two forces correspond to maximizing $\log \tilde{p}$ and minimizing $\log Z$. Several approximations to the negative phase are possible. Each of these approximations can be understood as making the negative phase computationally cheaper but also making it push down in the wrong locations.

### pp. 610 basic limitation of Contrastive Divergence

> Of course, CD is still an approximation to the correct negative phase. The main way that CD qualitatively fails to implement the correct negative phase is that it fails to suppress regions of high probability that are far from actual training examples. These regions that have high probability under the model but low probability under the data generating distribution are called spurious modes.

### pp. 611 theoretical justification of CD

Well I guess we can always justify something, given enough assumptions, etc.

> Bengio and Delalleau (2009) showed that CD can be interpreted as discarding the smallest terms of the correct MCMC update gradient, which explains the bias.

### pp. 612 overall intuition about CD and its side effects

> The CD algorithm can be thought of as penalizing the model for having a Markov chain that changes the input rapidly when the input comes from the data. This means training with CD somewhat resembles autoencoder training. Even though CD is more biased than some of the other training methods, it can be useful for pretraining shallow models that will later be stacked. This is because the earliest models in the stack are encouraged to copy more information up to their latent variables, thereby making it available to the later models. This should be thought of more of as an often-exploitable side effect of CD training rather than a principled design advantage.


### pp. 612 SML and PCD are the same.

> This approach was first discovered under the name stochastic maximum likelihood (SML) in the applied mathematics and statistics community (Younes, 1998) and later independently rediscovered under the name persistent contrastive divergence (PCD, or PCD-k to indicate the use of k Gibbs steps per update) in the deep learning community (Tieleman, 2008).

### pp. 614 all these methods can be combined with variational inference

> One key benefit to the MCMC-based methods described in this section is that they provide an estimate of the gradient of $\log Z$ , and thus we can essentially decompose the problem ... We can then use any other method to tackle $\log \tilde{p}$, and just add our negative phase gradient onto the other method's gradient. In particular, this means that our positive phase can make use of methods that provide only a lower bound.

Here it says "lower bound" in the end, instead of upper bound, is that because Variational methods can provide a lower bound for the positive phase, although upper bound should do in this case, as well.

## 18.3 Pseudolikelihood

Essentially, this is a method of approximating the true log likelihood (Eq. 18.19) for simplified computation. Top of pp. 616 shows that this method actually is a principled one asymptotically.

Eq. (18.21) gives generalized pseudolikelihood, which doesn't hack as much as the naive version.

### pp. 616 Pseudolikelihood methods depend on task and distribution structure.

> The performance of pseudolikelihood-based approaches depends largely on how the model will be used.

> poorly on tasks that require a good model of the full joint $p(x)$, .... However, it can perform better than maximum likelihood for tasks that require only the conditional distributions used during training, such as filling in small amounts of missing values.

> Generalized pseudolikelihood techniques are especially powerful if ... designed to capture the most important correlations while leaving out groups of variables that only have negligible correlation. For example, in natural images, pixels that are widely separated in space also have weak correlation, so the generalized pseudolikelihood can be applied with each S set being a small, spatially localized window.

### pp. 616 it can't be used with variational inference

suppose we use it, then it will be used to estimate the denominator in each of the conditional probability terms. Because we always assume that the unmarginalized log probability is easy to compute, and the marginalized one (usually over hidden variables) is difficult to compute, which should be used in the denominator.

## 18.4 Score Matching and Ratio Matching

For derivation why Eq. (18.22) and Eq. (18.25) are equivalent, check the original paper [Estimation of Non-Normalized Statistical Models by Score Matching](http://www.jmlr.org/papers/v6/hyvarinen05a.html). It's a brilliant usage of the boundary condition assumption. That is, for our model/data $p(x)$, when $x$ is so big, the probability of that is zero (well the exact assumption used is different from this, but intuition is the same).

Unfortunately, it can't be used with variational inference, since it needs gradient of log likelihood. This limits its usage a lot.

It can be extended to binary distribution, as shown in pp. 618, ratio matching.

I don't understand why the cost function for ratio matching works. Maybe, on avearge (flipping all bits), by making that ratio term (two $p$ terms) big, we can get consistent estimator.

## 18.5 Denoising Score Matching

It can be seen as regularization to simple score matching, under large model capacity and little data.

## 18.6 Noise-Contrastive Estimation

This is really clever.

Basic idea is, we turn unsupervised density estimation problem into supervised problem, by making model able to tell difference between data and noise. This is essentially the idea of Generative Adversarial Network.

There are multiple versions of it. Original version (NCE) uses some noise distribution totally independent from data. and it doesn't work in practice, although in theory (theory says it's consistent given enough data; see top of pp. 621; but maybe in practice the gradient is too small, or data is too little for theory to work).

### pp. 622

> When NCE is applied to problems with many random variables, it becomes less efficient. The logistic regression classifier can reject a noise sample by identifying any one variable whose value is unlikely. This means that learning slows down greatly after pmodel has learned the basic marginal statistics. Imagine learning a model of images of faces, using unstructured Gaussian noise as $p_{noise}$ . If $p_{model}$ learns about eyes, it can reject almost all unstructured noise samples without having learned anything about other facial features, such as mouths.

Later on, there is self-contrastive estimation, using model as noise. This is acutually equivalent to Maximum Likelihood!

### pp. 622 adversarial interpretation of maximum likelihood

> When the model distribution is copied to define a new noise distribution before each gradient step, NCE defines a procedure called self-contrastive estimation, whose expected gradient is equivalent to the expected gradient of maximum likelihood (Goodfellow, 2014).

I should say there should be some details that should be sorted out. Because if we naively use the probability of model as that for noise, and then according to Eq. (18.33), it would be 1/2, and no learning would be done, I think. But anyway, here we only need some intuition.

> The special case of NCE where the noise samples are those generated by the model suggests that maximum likelihood can be interpreted as a procedure that forces a model to constantly learn to distinguish reality from its own evolving beliefs, while noise contrastive estimation achieves some reduced computational cost by only forcing the model to distinguish reality from a fixed baseline (the noise model).

NCE (as well as self NCE) essentially uses logistic regression. When changing this classifier to a more powerful one (say CNN), we get GAN.

### pp. 623

> A closely related idea is that a good generative model should be able to generate samples that no classifier can distinguish from data. This idea yields generative adversarial networks (section 20.10.4).

## 18.7 Estimating the Partition Function

For derivation of annealed importance sampling, such as Eq. (18.53), please check Eq. (11.21) of PRML.

Eq. (11.21) of PRML tells you how to use importance weights to estimate the ratio of partition function for two distributions.

Here, two ones are $\tilde{p}$ (Eq. (18.54)) and $q$ (Eq. (18.60)). You can easily show that partition function for $\tilde{p}$ is that of $\tilde{p}_1$, which is what we want, and that of $q$ is 1. This can be shown easily because all $T, \tilde{T}$  are normalized.

Some other details for derivation of AIS:

1. $\eta_n$ and $1$ are the same, and $\eta_0$ and $0$ are the same. Remember this when reading Eq. (18.52) and some other equations.

### pp. 628 method can be popular due to popular papers, not due to its own merits.

> It is currently the most common way of estimating the partition function for undirected probabilistic models. The reasons for this may have more to do with the publication of an influential paper (Salakhutdinov and Murray, 2008) ... than with any inherent advantage the method has over the other method described below.

Other methods mentioned in this chapter are essentially modifications to AIS.

In the last they mentioned some cheap methods for estimating the partition function during training, as AIS is expensive.